In [174]:
from pyspark.sql import SparkSession
from datetime import date
import os
from pyspark.sql.functions import col, explode,monotonically_increasing_id

spark = SparkSession.builder.appName('pract').getOrCreate()
df_pyspark = spark.read.json('MPE1004.json')

In [175]:
df = df_pyspark.select(explode(col('results')))
df.show()

+--------------------+
|                 col|
+--------------------+
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0NBT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0xJT...|
|{true, {TVBFQ0NBT...|
|{true, {TVBFQ0xJT...|
+--------------------+
only showing top 20 rows



In [176]:
#df = df.withColumn('RowId', monotonically_increasing_id() + 1 )

In [177]:
df = df.withColumn('RowId', monotonically_increasing_id() + 1 )

In [178]:
df = df.selectExpr('RowId','col.id as itemId','col.sold_quantity','col.available_quantity')

In [179]:
visits_df = spark.read.option('header',True).csv('visits.csv')

In [180]:
#uter', 'leftouter', 'left', 'left_outer', 'rightouter', 'right', 'right_outer', 'leftsemi', 'left_semi', 'semi', 'leftanti', 'left_anti', 'anti', 'cross'.
visits_df.select('visits', 'itemId').show()

#visits_df.join(df,visits_df.itemId == df.itemId,
#              "right_outer").show()

+------+------------+
|visits|      itemId|
+------+------------+
|   203|MPE433108265|
|   170|MPE434382765|
|  1034|MPE433853177|
|  1772|MPE419883282|
|    33|MPE431714651|
|  1160|MPE438492919|
|  2669|MPE429448587|
|    36|MPE439307195|
|   257|MPE439307251|
|   292|MPE437503507|
|   102|MPE438828260|
|    29|MPE439307426|
|    50|MPE440306037|
|   120|MPE439307206|
|  2242|MPE431446248|
|   108|MPE439307250|
|    56|MPE439510012|
|   183|MPE439307317|
|   267|MPE439307286|
|    22|MPE439307385|
+------+------------+
only showing top 20 rows



In [165]:
rigth_df = df.select('RowId','sold_quantity','available_quantity')

In [166]:
df = df.join(visits_df,visits_df.itemId == df.itemId).select(df["*"],visits_df['visits'])

  (Conversiones u objetivos alcanzados / Total de usuarios) * 100

In [167]:
df = df.withColumn("conversionRate", round(col('sold_quantity')/col('visits'),4))

In [168]:
from pyspark.sql.functions import rank

In [169]:
from pyspark.sql.window import Window
windowSpec  = Window.orderBy(col("conversionRate").desc())
                                

In [170]:
df = df.withColumn("conversionRank", rank().over(windowSpec))

In [171]:
df.select('*').where(col('conversionRate')> 0.0).show()

22/06/13 01:20:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/13 01:20:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/13 01:20:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+------------+-------------+------------------+------+--------------+--------------+
|RowId|      itemId|sold_quantity|available_quantity|visits|conversionRate|conversionRank|
+-----+------------+-------------+------------------+------+--------------+--------------+
|    5|MPE431714651|           15|                 1|    33|        0.4545|             1|
|   34|MPE432291284|            2|                 1|     6|        0.3333|             2|
|   38|MPE427140390|           10|                 2|    81|        0.1235|             3|
|   26|MPE432439269|            2|                 1|    42|        0.0476|             4|
|    2|MPE434382765|            6|                 3|   170|        0.0353|             5|
|    1|MPE433108265|            6|                 9|   203|        0.0296|             6|
|   36|MPE433252062|            2|                 1|    92|        0.0217|             7|
|   33|MPE433933924|            1|                 1|    49|        0.0204|             8|

22/06/13 01:20:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [182]:
df = df.withColumn('stockPercentage', (col('available_quantity')*100)/(col('sold_quantity')+col('available_quantity')))

In [184]:
df.select('ItemId','available_quantity','stockPercentage').show(1000)

+------------+------------------+------------------+
|      ItemId|available_quantity|   stockPercentage|
+------------+------------------+------------------+
|MPE433108265|                 9|              60.0|
|MPE434382765|                 3|33.333333333333336|
|MPE433853177|                17|              85.0|
|MPE419883282|                18| 54.54545454545455|
|MPE431714651|                 1|              6.25|
|MPE438492919|               100|             100.0|
|MPE429448587|                50|             100.0|
|MPE439307195|                 3|             100.0|
|MPE439307251|                 3|             100.0|
|MPE437503507|                10|             100.0|
|MPE438828260|                 3|             100.0|
|MPE439307426|                 3|             100.0|
|MPE440306037|                 1|             100.0|
|MPE439307206|                 3|             100.0|
|MPE431446248|                23|              92.0|
|MPE439307250|                 3|             

In [138]:
from pyspark.sql import SparkSession
from datetime import date
import os
from pyspark.sql.functions import col, explode,monotonically_increasing_id

spark = SparkSession.builder.appName('pract').getOrCreate()
df_pyspark = spark.read.json('MPE1004.json')
df = df_pyspark.select(explode(col('results')))
df = df.withColumn('RowId', monotonically_increasing_id() + 1 )
df = df.selectExpr('RowId','col.id as itemId','col.sold_quantity','col.available_quantity')
visits_df = spark.read.option('header',True).csv('visits.csv')
rigth_df = df.select('RowId','sold_quantity','available_quantity')
df = df.join(visits_df,visits_df.itemId == df.itemId).select(df["*"],visits_df['visits']).show()

+-----+------------+-------------+------------------+------+
|RowId|      itemId|sold_quantity|available_quantity|visits|
+-----+------------+-------------+------------------+------+
|    1|MPE433108265|            6|                 9|   203|
|    2|MPE434382765|            6|                 3|   170|
|    3|MPE433853177|            3|                17|  1034|
|    4|MPE419883282|           15|                18|  1772|
|    5|MPE431714651|           15|                 1|    33|
|    6|MPE438492919|            0|               100|  1160|
|    7|MPE429448587|            0|                50|  2669|
|    8|MPE439307195|            0|                 3|    36|
|    9|MPE439307251|            0|                 3|   257|
|   10|MPE437503507|            0|                10|   292|
|   11|MPE438828260|            0|                 3|   102|
|   12|MPE439307426|            0|                 3|    29|
|   13|MPE440306037|            0|                 1|    50|
|   14|MPE439307206|    

In [139]:
from pyspark.sql import SparkSession
from datetime import date
import os
from pyspark.sql.functions import rank
from pyspark.sql.window import Window
from pyspark.sql.functions import col, explode,monotonically_increasing_id

spark = SparkSession.builder.appName('pract').getOrCreate()
df_pyspark = spark.read.json('MPE1004.json')
df = df_pyspark.select(explode(col('results')))
df = df.withColumn('RowId', monotonically_increasing_id() + 1 )
df = df.selectExpr('RowId','col.id as itemId','col.sold_quantity','col.available_quantity')
visits_df = spark.read.option('header',True).csv('visits.csv')
rigth_df = df.select('RowId','sold_quantity','available_quantity')
df = df.join(visits_df,visits_df.itemId == df.itemId).select(df["*"],visits_df['visits'])
windowSpec  = Window.orderBy(col("conversionRate").desc())
df = df.withColumn("conversionRate", round(col('sold_quantity')/col('visits'),4))
df = df.withColumn("conversionRank", rank().over(windowSpec))
df.select('*').where(col('conversionRate')> 0.0).show()

22/06/13 01:16:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/13 01:16:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/13 01:16:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+------------+-------------+------------------+------+--------------+--------------+
|RowId|      itemId|sold_quantity|available_quantity|visits|conversionRate|conversionRank|
+-----+------------+-------------+------------------+------+--------------+--------------+
|    5|MPE431714651|           15|                 1|    33|        0.4545|             1|
|   34|MPE432291284|            2|                 1|     6|        0.3333|             2|
|   38|MPE427140390|           10|                 2|    81|        0.1235|             3|
|   26|MPE432439269|            2|                 1|    42|        0.0476|             4|
|    2|MPE434382765|            6|                 3|   170|        0.0353|             5|
|    1|MPE433108265|            6|                 9|   203|        0.0296|             6|
|   36|MPE433252062|            2|                 1|    92|        0.0217|             7|
|   33|MPE433933924|            1|                 1|    49|        0.0204|             8|

22/06/13 01:16:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [140]:
from pyspark.sql import SparkSession
from datetime import date
import os
from pyspark.sql.functions import rank
from pyspark.sql.window import Window
from pyspark.sql.functions import col, explode,monotonically_increasing_id

spark = SparkSession.builder.appName('pract').getOrCreate()
df_pyspark = spark.read.json('MPE1004.json')
df = df_pyspark.select(explode(col('results')))
df = df.withColumn('RowId', monotonically_increasing_id() + 1 )
df = df.selectExpr('RowId','col.id as itemId','col.sold_quantity','col.available_quantity')
visits_df = spark.read.option('header',True).csv('visits.csv')
rigth_df = df.select('RowId','sold_quantity','available_quantity')
df = df.join(visits_df,visits_df.itemId == df.itemId).select(df["*"],visits_df['visits'])
df = df.withColumn('stockPercentage', (col('available_quantity')*100)/(col('sold_quantity')+col('available_quantity')))
df.select('id','available_quantity','stockPercentage').show(1000)

AnalysisException: cannot resolve 'id' given input columns: [RowId, available_quantity, itemId, sold_quantity, stockPercentage, visits];
'Project ['id, available_quantity#2889L, stockPercentage#2934]
+- Project [RowId#2883L, itemId#2886, sold_quantity#2888L, available_quantity#2889L, visits#2911, (cast((available_quantity#2889L * cast(100 as bigint)) as double) / cast((sold_quantity#2888L + available_quantity#2889L) as double)) AS stockPercentage#2934]
   +- Project [RowId#2883L, itemId#2886, sold_quantity#2888L, available_quantity#2889L, visits#2911]
      +- Join Inner, (itemId#2910 = itemId#2886)
         :- Project [RowId#2883L, col#2881.id AS itemId#2886, col#2881.sold_quantity AS sold_quantity#2888L, col#2881.available_quantity AS available_quantity#2889L]
         :  +- Project [col#2881, (monotonically_increasing_id() + cast(1 as bigint)) AS RowId#2883L]
         :     +- Project [col#2881]
         :        +- Generate explode(results#2868), false, [col#2881]
         :       

In [185]:
from pyspark.sql import SparkSession
from datetime import date
import os
from pyspark.sql.functions import rank
from pyspark.sql.window import Window
from pyspark.sql.functions import col, explode,monotonically_increasing_id

spark = SparkSession.builder.appName('pract').getOrCreate()
df_pyspark = spark.read.json('MPE1004.json')
df = df_pyspark.select(explode(col('results')))
df = df.withColumn('RowId', monotonically_increasing_id() + 1 )
df = df.selectExpr('RowId','col.id as itemId','col.sold_quantity','col.available_quantity')
visits_df = spark.read.option('header',True).csv('visits.csv')
rigth_df = df.select('RowId','sold_quantity','available_quantity')
df = df.join(visits_df,visits_df.itemId == df.itemId).select(df["*"],visits_df['visits'])
df = df.withColumn('stockPercentage', (col('available_quantity')*100)/(col('sold_quantity')+col('available_quantity')))
df.select('ItemId','available_quantity','stockPercentage').show(1000)

+------------+------------------+------------------+
|      ItemId|available_quantity|   stockPercentage|
+------------+------------------+------------------+
|MPE433108265|                 9|              60.0|
|MPE434382765|                 3|33.333333333333336|
|MPE433853177|                17|              85.0|
|MPE419883282|                18| 54.54545454545455|
|MPE431714651|                 1|              6.25|
|MPE438492919|               100|             100.0|
|MPE429448587|                50|             100.0|
|MPE439307195|                 3|             100.0|
|MPE439307251|                 3|             100.0|
|MPE437503507|                10|             100.0|
|MPE438828260|                 3|             100.0|
|MPE439307426|                 3|             100.0|
|MPE440306037|                 1|             100.0|
|MPE439307206|                 3|             100.0|
|MPE431446248|                23|              92.0|
|MPE439307250|                 3|             